### 07-03. 깊은 CNN으로 MNIST 분류하기 

앞서 배운 챕터에서 1번 레이어와 2번 레이어는 동일하되, 

새로운 합성곱층과 전결합층을 추가 

#### 1번 레이어: 합성곱층(Convolutional layer)
합성곱(in_channel = 1, out_channel = 32, kernel_size = 3, stride = 1, padding = 1) + 활성화 함수 ReLU \
맥스풀링(kernel_size = 2, stride = 2)

#### 2번 레이어: 합성곱층(Convolutional layer)
합성곱(in_channel = 32, out_channel = 64, kernel_size = 3, stride = 1, padding = 1) + 활성화 함수 ReLU 
맥스풀링(kernel_size = 2, stride = 2)

#### 3번 레이어: 합성곱층(Convolutional layer)
합성곱(in_channel = 64, out_channel = 128, kernel_size = 3, stride = 1, padding = 1) + 활성화 함수 ReLU 
맥스풀링(kernel_size = 2, stride = 2, padding = 1)

#### 4번 레이어: 전결합층 (Fully-Connected layer)
특성맵을 펼친다. # batch_size * 4 * 4 * 128 --> batch_size * 2048
전결합층(뉴런 625개) + 활성화 함수 ReLU 

#### 5번 레이어: 전결합층 (Fully-Connected layer)
전결합층(뉴런 10개) + 활성화 함수 Softmax

In [1]:
import torch 
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import torch.nn.init

device = 'cuda' if torch.cuda.is_available() else 'cpu'

# 랜덤 시드 고정 
torch.manual_seed(777)

# GPU 사용 가능일 경우 랜덤 시드 고정 
if device == 'cuda': 
    torch.cuda.manual_seed_all(777)

# 학습에 사용할 파라미터 설정
learning_rate = 0.001
training_epochs = 15
batch_size = 100

# 데이터셋 정의
mnist_train = dsets.MNIST(root='MNIST_data\\', # 다운로드 경로 지정 
                          train = True,       # True를 지정하면 훈련 데이터로 다운로드
                          transform = transforms.ToTensor(), # 텐서로 변환
                          download = True)

mnist_test = dsets.MNIST(roots = 'MNIST_data\\', 
                         train = False, 
                         transform = transforms.ToTensor(), 
                         download = True)

data_loader = torch.utils.data.DataLoader(dataset = mnist_train, 
                                          batch_size = batch_size, 
                                          shuffle = True, 
                                          drop_last = True)

class CNN(torch.nn.Module): 
    def __init__(self): 
        super(CNN, self).__init__()
        self.keep_prob = 0.5
        
        self.layer1 = torch.nn.Sequential(
            torch.nn.Conv2d(1, 32, kernel_size = 3, stride = 1, padding = 1),
            torch.nn.ReLU(), 
            torch.nn.MaxPool2d(kernel_size = 2, strid = 2)
        )

        self.layer2 = torch.nn.Sequential(
            torch.nn.Conv2d(32, 64, kernel_size = 3, stride = 1, padding = 1), 
            torch.nn.ReLU(), 
            torch.nn.MaxPool2d(kernel_size = 2, stride = 2)
        )

        self.layer3 = torch.nn.Sequential(
            torch.nn.Conv2d(64, 128, kernel_size = 3, stride = 1, padding = 1), 
            torch.nn.ReLU(), 
            torch.nn.MaxPool2d(kernel_size = 2, stride = 2, padding = 1)
        )

        self.fc1 = torch.nn.Linear(4 * 4 * 128, 625, bias = True)
        torch.nn.init.xavier_uniform_(self.fc1.weight)
        self.layer4 = torch.nn.Sequential(
            self.fc1, 
            torch.nn.ReLU(), 
            torch.nn.Dropout(p = 1 - self.keep_prob)
        )
        self.fc2 = torch.nn.Linear(625, 10, bias = True)
        torch.nn.init.xavier_uniform_(self.fc2.weight)

    def forward(self, x): 
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)
        out = out.view(out.size(0), -1) # Flatten them for FC
        out = self.layer4(out)
        out = self.fc2(out)
        return out 

# CNN 모델 정의 
model = CNN().to(device)

criterion = torch.nn.CrossEntropyLoss().to(device)  # 비용 함수에 소프트맥스 함수 포함되어져 있음 
optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)

total_batch = len(data_loader)
print('총 배치의 수: {}'.format(total_batch))

for epoch in range(training_epochs): 
    avg_cost = 0

    for X, Y in data_loader:     # 미니 배치 단위로 꺼내온다. X는 미니 배치, Y는 레이블. 
        X = X.to(device)
        Y = Y.to(device)

        optimizer.zero_grad()
        hypothesis = model(X)
        cost = criterion(hypothesis, Y)
        cost.backward()
        optimizer.step()

        avg_cost += cost / total_batch

    print('[Epoch: {:>4}] cost = {:>.9}'.format(epoch + 1, avg_cost))

with torch.no_grad(): 
    X_test = mnist_test.test_data.view(len(mnist_test), 1, 28, 28).float().to(device)
    Y_test = mnist_test.test_labels.to(device)

    prediction = model(X_test)
    correct_prediction = torch.argmax(prediction, 1) == Y_test
    accuracy = correct_prediction.float().mean()
    print('Accuracy: ', accuracy.item())


RuntimeError: Dataset not found. You can use download=True to download it